# Hyperparameter Tuning for Logistic Regression with Boosting Model

## Introduction

This notebook aims to optimize the performance of the Logistic Regression with boosting model applied to the League of Legends matches dataset. We will compare different hyperparameter tuning methods: Grid Search, Randomized Search, and Bayesian Optimization.

## Installing Prerequisites


In [1]:
# !pip install catboost
# !pip install scikit-optimize
# !pip install category_encoders
# !pip install bayesian-optimization

## Importing Libraries

In [2]:
import time
import pickle
import numpy as np
import pandas as pd
import category_encoders as ce
import matplotlib.pyplot as plt
from skopt import BayesSearchCV
from catboost import CatBoostClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score

## Loading Data

First, we load the dataset and split it into a training and testing sets.

In [3]:
#Loading Dataset
file_path = '../../sentiment_scores.csv'
df = pd.read_csv(file_path)

#Splitting into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

## Data Preprocessing

We preprocess the data using One-Hot Encoding and Target Encoding.

One-Hot Encoding the low cardinality variables, namely, 'League',  'Season' and 'Type'.

In [4]:
# One-Hot Encoding
train_df_onehot = pd.get_dummies(train_df, columns=['League', 'Season', 'Type'])
test_df_onehot = pd.get_dummies(test_df, columns=['League', 'Season', 'Type'])
missing_cols = set(train_df_onehot.columns) - set(test_df_onehot.columns)

# Handling missing columns
for c in missing_cols:
    test_df_onehot[c] = 0
test_df_onehot = test_df_onehot[train_df_onehot.columns]

Applying Target Encoding to the high cardinality variables.

In [5]:
# Target Encoding
target_cols = [
    'blueTop', 'blueJungle', 'blueMiddle', 'blueADC', 'blueSupport',
    'redTop', 'redJungle', 'redMiddle', 'redADC', 'redSupport',
    'blueTopChamp', 'blueJungleChamp', 'blueMiddleChamp', 'blueADCChamp', 'blueSupportChamp',
    'redTopChamp', 'redJungleChamp', 'redMiddleChamp', 'redADCChamp', 'redSupportChamp',
    'blueTeamTag', 'redTeamTag'
]
target_variable = 'bResult'
encoder = ce.TargetEncoder(cols=target_cols)
encoder.fit(train_df, train_df[target_variable])
train_df_target_encoded = encoder.transform(train_df)
test_df_target_encoded = encoder.transform(test_df)

Combining the One-Hot and Target Encoded Dataframes for training and testing sets.

In [6]:
# Removing Target Encoded Columns from One-Hot Encoded DataFrame
train_df_onehot = train_df_onehot.drop(columns=target_cols, axis=1)
test_df_onehot = test_df_onehot.drop(columns=target_cols, axis=1)

# Concatenating One-Hot and Target Encoded DataFrames:
train_df_encoded = pd.concat([train_df_onehot, train_df_target_encoded[target_cols]], axis=1)
test_df_encoded = pd.concat([test_df_onehot, test_df_target_encoded[target_cols]], axis=1)

## Feature Selection

We perform feature selection using averaged importance scores from mutual information and CatBoost.

In [7]:
# Separate features and target variable
X_train = train_df_encoded.drop([target_variable], axis=1)
y_train = train_df_encoded[target_variable]

# Calculate Mutual Information scores
mi_scores = mutual_info_classif(X_train, y_train)
mi_scores = pd.Series(mi_scores, name='MI_Scores', index=X_train.columns)

# CatBoost Importance
catboost_model = CatBoostClassifier(iterations=100, verbose=0)
catboost_model.fit(X_train, y_train)
catboost_importances = pd.Series(catboost_model.get_feature_importance(), name='CatBoost_Importance', index=X_train.columns)

# Combine and Normalize
importance_df = pd.concat([mi_scores, catboost_importances], axis=1)
importance_df['MI_Scores'] = (importance_df['MI_Scores'] - importance_df['MI_Scores'].min()) / (importance_df['MI_Scores'].max() - importance_df['MI_Scores'].min())
importance_df['CatBoost_Importance'] = (importance_df['CatBoost_Importance'] - importance_df['CatBoost_Importance'].min()) / (importance_df['CatBoost_Importance'].max() - importance_df['CatBoost_Importance'].min())
importance_df['Combined_Importance'] = (importance_df['MI_Scores'] + importance_df['CatBoost_Importance']) / 2

# Sort and Select Features
sorted_features = importance_df.sort_values(by='Combined_Importance', ascending=False).index
N = 22
selected_features = sorted_features[:N]
X_train_selected = X_train[selected_features]
X_test_selected = test_df_encoded.drop([target_variable], axis=1)[selected_features]

## Hyperparameter Tuning

We use three methods for hyperparameter tuning: Grid Search, Randomized Search, and Bayesian Optimization.

In [8]:
# Training the model to prepare for Hyperparameter Tuning
def train_evaluate_lr(params):
    lr_clf = AdaBoostClassifier(
        estimator=LogisticRegression(),
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        random_state=42
    )
    lr_clf.fit(X_train_selected, y_train)
    y_pred = lr_clf.predict(X_test_selected)
    return accuracy_score(y_test, y_pred)

In [9]:
# Grid Search
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0]
}

grid_search = GridSearchCV(AdaBoostClassifier(estimator=LogisticRegression(), random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_selected, y_train)
grid_best_params = grid_search.best_params_
grid_best_accuracy = grid_search.best_score_

In [10]:
# Randomized Search
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0]
}

random_search = RandomizedSearchCV(AdaBoostClassifier(estimator=LogisticRegression(), random_state=42), param_dist, n_iter=9, cv=5, scoring='accuracy')
random_search.fit(X_train_selected, y_train)
random_best_params = random_search.best_params_
random_best_accuracy = random_search.best_score_

In [11]:
# Bayesian Optimization
y_test = test_df_encoded[target_variable]

def lr_bayesian(n_estimators, learning_rate):
    params = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate
    }
    return train_evaluate_lr(params)

optimizer = BayesianOptimization(
    f=lr_bayesian,
    pbounds={
        'n_estimators': (50, 200),
        'learning_rate': (0.01, 1.0)
    },
    random_state=42
)

optimizer.maximize(init_points=5, n_iter=15)
bayesian_best_params = optimizer.max['params']
bayesian_best_accuracy = optimizer.max['target']

|   iter    |  target   | learni... | n_esti... |
-------------------------------------------------
| 1         | 0.6319    | 0.3808    | 192.6     |
| 2         | 0.6346    | 0.7347    | 139.8     |
| 3         | 0.5897    | 0.1645    | 73.4      |
| 4         | 0.5871    | 0.0675    | 179.9     |
| 5         | 0.6346    | 0.6051    | 156.2     |
| 6         | 0.6306    | 0.8111    | 139.8     |
| 7         | 0.5541    | 0.01      | 139.6     |
| 8         | 0.6319    | 0.9394    | 156.5     |
| 9         | 0.6319    | 0.424     | 156.7     |
| 10        | 0.562     | 0.04398   | 156.3     |
| 11        | 0.6332    | 0.8932    | 156.0     |
| 12        | 0.6306    | 0.7302    | 157.1     |
| 13        | 0.6214    | 0.2352    | 157.3     |
| 14        | 0.6306    | 0.7197    | 157.1     |
| 15        | 0.624     | 0.3065    | 192.0     |
| 16        | 0.6346    | 0.9023    | 156.0     |
| 17        | 0.6319    | 1.0       | 155.6     |
| 18        | 0.6332    | 0.9431    | 192.4     |


## Evaluating Models

After hyperparameter tuning, we evaluate the best models from each method on the test data.

In [12]:
bayesian_best_params = {key: int(value) if key == 'n_estimators' else value for key, value in bayesian_best_params.items()}

# Function to train and evaluate model
def train_evaluate(params, X_train, y_train, X_test, y_test, method):
    # Extract logistic regression-specific and AdaBoost-specific parameters
    lr_params = {key: params[key] for key in ['penalty', 'solver'] if key in params}
    ada_params = {key: params[key] for key in ['n_estimators', 'learning_rate'] if key in params}
    
    # Create logistic regression classifier
    logistic_clf = LogisticRegression(
        random_state=42, 
        **lr_params
    )
        
    # Create AdaBoost classifier
    ada_clf = AdaBoostClassifier(
        estimator=logistic_clf, 
        random_state=42, 
        **ada_params
    )
    
    # Training
    start_train_time = time.time()
    ada_clf.fit(X_train, y_train)
    end_train_time = time.time()
    
    # Testing
    start_test_time = time.time()
    y_pred = ada_clf.predict(X_test)
    end_test_time = time.time()
    
    # Scoring
    test_accuracy = accuracy_score(y_test, y_pred)
    test_precision = precision_score(y_test, y_pred)
    test_recall = recall_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred)
    
    train_time = end_train_time - start_train_time
    test_time = end_test_time - start_test_time
    
    results_df = pd.DataFrame({
        'Method': [method],
        'Accuracy': [test_accuracy],
        'Precision': [test_precision],
        'Recall': [test_recall],
        'F1-Score': [test_f1],
        'Training Time (s)': [train_time],
        'Prediction Time (s)': [test_time]
    })
    
    return results_df

In [13]:
# Prepare the test data
X_test_selected = test_df_encoded.drop([target_variable], axis=1)[selected_features]
y_test = test_df_encoded[target_variable]

# Convert Bayesian best parameters to the correct format
bayesian_best_params = {key: int(value) if isinstance(value, float) and value.is_integer() else value for key, value in bayesian_best_params.items()}

# Evaluate the model using the best parameters from each method
results_grid = train_evaluate(grid_best_params, X_train_selected, y_train, X_test_selected, y_test, 'Grid Search')
results_random = train_evaluate(random_best_params, X_train_selected, y_train, X_test_selected, y_test, 'Random Search')
results_bayesian = train_evaluate(bayesian_best_params, X_train_selected, y_train, X_test_selected, y_test, 'Bayesian Optimization')


## Results Comparison

Comparing the performance of the models using the best hyperparameters from Grid Search, Randomized Search, and Bayesian Optimization methods.

In [14]:
# Concatenate the results
final_results = pd.concat([results_grid, results_random, results_bayesian], axis=0).reset_index(drop=True)

# Display the best parameters
print(f"Grid Search - Best Params: {grid_best_params}, Best Accuracy: {grid_best_accuracy}")
print(f"Random Search - Best Params: {random_best_params}, Best Accuracy: {random_best_accuracy}")
print(f"Bayesian Optimization - Best Params: {bayesian_best_params}, Best Accuracy: {bayesian_best_accuracy}")

# Display the final results
print(final_results)

Grid Search - Best Params: {'learning_rate': 1.0, 'n_estimators': 200}, Best Accuracy: 0.7003814465104788
Random Search - Best Params: {'n_estimators': 200, 'learning_rate': 1.0}, Best Accuracy: 0.7003814465104788
Bayesian Optimization - Best Params: {'learning_rate': 0.7346740023932911, 'n_estimators': 139}, Best Accuracy: 0.6345646437994723
                  Method  Accuracy  Precision    Recall  F1-Score  \
0            Grid Search  0.630607   0.652838  0.711905  0.681093   
1          Random Search  0.630607   0.652838  0.711905  0.681093   
2  Bayesian Optimization  0.634565   0.651163  0.733333  0.689810   

   Training Time (s)  Prediction Time (s)  
0           3.300578             0.080008  
1           3.666663             0.069997  
2           2.536727             0.049999  
